# 한글 논문 Fasttext 분석

In [ ]:
# %% PDF reader 

from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
 
path = "D:/work/paper_ko/datasets/"
texts = '' 
for i in os.listdir(path):
    # print(i)
    
    if '.pdf' in i:
        text = '\n' + read_pdf_PDFMINER(path + i)
    
# %%

def read_pdf_PDFMINER(pdf_file_path):
    """
    pdf_file_path: 'dir/aaa.pdf'로 구성된 path로부터 
    내부의 text 파일을 모두 읽어서 스트링을 리턴함.
    https://pdfminersix.readthedocs.io/en/latest/tutorials/composable.html
    """
    output_string = StringIO()
    with open(pdf_file_path, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return str(output_string.getvalue())

In [ ]:
# %% stopwords 

f = open(path + 'stopwords.txt', mode='r', encoding='utf-8')

lines = f.readlines()

stop_words = ''
for ll in lines:
    # print(ll)
    stop_words += ' ' + ll

In [ ]:
## 특수문자 처리
import re

parse_text = text

clean_lines = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', parse_text)
p = re.compile('[^0-9]')
clean_lines = "".join(p.findall(clean_lines))



In [ ]:
# 이상한 데이터가 많아서 
# 한번 더 처리

lines = [] 
for cc in clean_lines.split('\n\n'):
    print(cc)
    if cc != '\n' and cc != '\n\n' and cc != ' ' and cc != '  ' and cc != '   ':
        lines.append(cc)



In [ ]:

from konlpy.tag import Okt

okt = Okt() 
results = []
for ll in lines:
    results.append(okt.nouns(ll))

In [ ]:

from ckonlpy.tag import Twitter
twitter = Twitter()
# tt = twitter.morphs("테스트를 해본다")


results = []

for cc in clean_lists:
    t_cc = twitter.morphs(cc)
    
    for tt in token_text:
        
        if tt not in stop_words:
            results.append(tt)


In [ ]:
from gensim.models import FastText
ft_model = FastText(results, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
print(ko_model.wv.most_similar('멘토', topn=10))
print(ft_model.wv.similarity('멘토링', '창업'))

In [ ]:
# PCA 를 통한 시각화 


from sklearn.decomposition import PCA

words = [
    '멘토링',
    '멘토',
    '멘티',
    '창업',
    '취업',
    '기업가정신'
]
# matplotplib 폰트 설정을 안해서 그래프에서 한국어 라벨이 깨져서 아래 단어로 임시처리했습니다. ㅠ
word_labels = [
    'mentoring',
    'mento',
    'menti',
    'startup',
    'job', 
    'entrepreneurship'
]
pca = PCA(n_components=2)
xys = pca.fit_transform([ft_model.wv.word_vec(w) for w in words])
xs = xys[:,0]
ys = xys[:,1]
plt.figure(figsize=(14, 10))
plt.scatter(xs, ys, marker='o')
for i, v in enumerate(word_labels):
    plt.annotate(v, xy=(xs[i], ys[i]))